# Convolutional Neural Network Training using Keras on Google Cloud Platform

# Overview

Google Cloud Platform (GCP) is a suite of cloud computing services that runs on the same infrastructure that Google uses internally for its end-user products. Alongside a set of management tools, it provides infrastructure as a service, platform as a service, and serverless computing environments for various applications such as computing, data storage, data analytics and machine learning. In this assigment, we are going to set up a Jupyter Notebook on Google Cloud Platform and use Keras to build and train a Convolutional Neural Network (CNN) on the MNIST dataset.

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled $28 \times 28$ pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). 

## Setup

1. First, we need to create a free account on Google Cloud Platform and create a new project as shown in the screenshot below.

> ![New Project in GCP](https://raw.githubusercontent.com/zhoujc999/CS189-Final-Project-T/master/Assets/Screenshot_4.png)

---
2. We need to create a VM instance to run our Jupyter notebook on the cloud. To do this, click on the compute option and select ‘VM Instances’.

> ![New Project in GCP](https://raw.githubusercontent.com/zhoujc999/CS189-Final-Project-T/master/Assets/Screenshot_5.png)

---
3. After setting up the VM instance according to your preference, we are ready to tweak the firewall settings to enable us to access our Jupyter notebook on our local browser. Note: Select the 2 options as shown below to allow SSH connections.

> ![New Project in GCP](https://raw.githubusercontent.com/zhoujc999/CS189-Final-Project-T/master/Assets/Screenshot_6.png)

---
4. Under 'VPC network', click on 'Firewall' to add firewall rules to our newly created VM instance. Under 'Source IP Ranges', input `0.0.0.0/0` to allow connections from all IPs. Under 'Protocol and Ports', input your desired port for your browser to connect to the Jupyter notebook on Google Cloud Platform. Click on create after you are satisfied with the details.

> ![New Project in GCP](https://raw.githubusercontent.com/zhoujc999/CS189-Final-Project-T/master/Assets/Screenshot_7.png)

---
5. We need to return to the VM instances interface to start our VM instance. Take note of the external IP address.

> ![New Project in GCP](https://raw.githubusercontent.com/zhoujc999/CS189-Final-Project-T/master/Assets/Screenshot_8.png)

---
6. After connecting to the VM instance via SSH, we need to install Jupyter notebook. In the console, type the following.
```shell
> apt install jupyter
```

---
7. Once the relevant packages are installed, we want to install PyTorch, which contains the building blocks for building our convolutional neural network.
```shell
> pip3 install keras
```

---
8. Next, we need to configure our Jupyter notebook to accept connections from our browser. After we generate the configuration file, we add the following lines of code in `~/.jupyter/jupyter_notebook_config.py`. Note: `<PORT_NUMBER>` is the port you've chosen for the firewall settings.
```shell
> jupyter notebook --generate-config
```
```python
c = get_config()
c.NotebookApp.ip = '*'
c.NotebookApp.open_browser = False
c.NotebookApp.port = <PORT_NUMBER>
```

---
9. Finally, we are able to launch the Jupyter notebook with the following command. Connect to the notebook from our browser by going to the following address: `http://<EXTERNAL_IP_ADDRESS>:<PORT_NUMBER>`
```shell
jupyter-notebook --no-browser --port=<PORT_NUMBER>
```

Now, we have successfully set-up our Jupyter notebook on Google Cloud Platform and we are ready to build and train our CNN classifier.

## Data

### Getting the *data*

First, we need to download the MNIST dataset with the following code.

```python
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path="mnist.npz")
```

Loading the dataset returns four NumPy arrays:

- The train_images and train_labels arrays are the training set—the data the model uses to learn.
- The model is tested against the test set, the test_images, and test_labels arrays.

The images are $28 \times 28$ NumPy arrays, with pixel values ranging from 0 to 255. The labels are an array of integers, ranging from 0 to 9, corresponding the digits.

### Preprocess the data

Before we feed the image data into the neural network, we need to preprocess the data to enable the network to learn better. One such procedure is to scale these values to a range of 0 to 1 before feeding them to the neural network model. To do so, divide the values by 255. It's important that the training set and the testing set be preprocessed in the same way.

```python
train_images = train_images / 255.0

test_images = test_images / 255.0
```

## Convolution Neural Network

### Build the model

You will be writing some code to build the neural network and configure the layers of the model. The basic building block of a neural network is the layer. Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Similar to the assignment using PyTorch on AWS, we suggest the following architecture:

1. A convolutional layer with 10 output channels and kernel size of 5
2. A max pool layer with kernel of size 2 and stride of 2
3. A ReLU activation layer
4. A convolutional layer with 20 output channels and kernel size of 5
5. A dropout layer with probability 0.5
6. A max pool layer with stride equals 2
7. A ReLU activation layer
8. 2 fully connected layers with a dropout layer in between
9. A softmax output layer

```python
from tensorflow.keras import Sequential, layers

model = Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Conv2D(10, 5),
    layers.MaxPooling2D(),
    layers.Activation(activations.relu),
    layers.Conv2D(20, 5),
    layers.Dropout(0.5),
    layers.MaxPooling2D(),
    layers.Activation(activations.relu),
    layers.Dropout(0.5),
    layers.Dense(50),
    tf.keras.layers.Dense(10),
    layers.Softmax()
])
```

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

- Loss function: This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction. We will use the sparse categorical cross-entropy loss as it is the most suitable for our problem.
- Optimizer: This is how the model is updated based on the data it sees and its loss function. In this case, we will use the Adam optimizer, which has proved to be one of best-performing optimizer under most conditions.
- Metrics: Used to monitor the training and testing steps. We use accuracy, the fraction of the images that are correctly classified.

```python
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
```

### Train the model


Training the neural network model requires the following steps:

1. Feed the training data to the model. In this example, the training data is in the train_images and train_labels arrays.
2. The model learns to associate images and labels.

To start training, call the `model.fit` method — so called because it "fits" the model to the training data:

```python
model.fit(train_images, train_labels, epochs=10)
```

### Evaluate the accuracy

Next, compare how the model performs on the test dataset:

```python
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
```

Report the training and test accuracy:

Training accuracy = _________

Test accuracy = _________

### Make predictions

With the model trained, you can use it to make predictions about some images. For example, we can make a prediction of the first image in `test_images`.

```python
predictions = probability_model.predict(test_images)
np.argmax(predictions[0])
```

Report the prediction of the first image:

Prediction = _________
